In [ ]:
import numpy as np
import bornagain as ba
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [ ]:
DATA_FILE = "/data/herck/Experiments/Asma/Alhroob_JX_37_48_13670.tif"

In [ ]:
def rebin_linear(a, newshape):
    '''Rebin an array using linear interpolation
    '''
    assert len(a.shape) == len(newshape)

    avals = a.flatten()
    slices_old = [ slice(0, old) for old in a.shape ]
    slices_new = [ slice(0, old-1, new*1j) for old, new in zip(a.shape, newshape) ]
    inds_in = np.mgrid[slices_old].reshape(len(a.shape), -1).T
    inds_out = np.mgrid[slices_new].reshape(len(a.shape), -1).T

    result = griddata(inds_in, avals, inds_out)
    return result.reshape(newshape)

In [ ]:
def normalize(a, mean, std):
    m = a.mean()
    s = np.std(a)
    normal_a = (a-m)/s
    return std*normal_a + mean

In [ ]:
data = ba.IntensityDataIOFactory.readOutputData(DATA_FILE)
data_arr = np.log(data.getArray().clip(0.) +1)
plt.imshow(data_arr)
plt.show()

In [ ]:
data_arr.shape

In [ ]:
mean = data_arr.mean()
std = np.std(data_arr)
print("Mean: {0}, std: {1}".format(mean, std))

In [ ]:
cropped = data_arr[:634, 262:936]
cropped.shape

In [ ]:
data_out = rebin_linear(cropped, (224,224))

In [ ]:
data_norm = normalize(data_out, -0.2, 1.0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data_norm)
plt.show()

In [ ]:
np.std(data_norm)

In [ ]:
np.save("./realdata", data_norm)